<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/AI_Forced_Alignment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intel-tensorflow is required to run on CPU
!pip install torch torchaudio soundfile omegaconf intel-tensorflow tensorflow_hub

     |████████████████████████████████| 7.6MB 4.8MB/s 
     |████████████████████████████████| 386.6MB 40kB/s 
     |████████████████████████████████| 645kB 41.6MB/s 
     |████████████████████████████████| 460kB 32.4MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: torchaudio 0.7.2 has requirement torch==1.7.1, but you'll have torch 1.7.0+cu101 which is incompatible.
ERROR: intel-tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [15]:
#speech_file = 'speech_orig.wav'
speech_file = '/content/recorded.wav'


import os
import torch
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

language = 'en'

# load provided utils using torch.hub for brevity
_, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

# see available models
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

# load the actual tf model
tf_model = tf_hub.load(models.stt_models.get(language).latest.tf)

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file(models.stt_models.get(language).latest.meta.sample, dst = speech_file, progress=True)
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master



the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [20]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))

you would not believe your eyeswith ten million firefes


In [18]:
batches

[['/content/recorded.wav']]

In [3]:
tf_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7ff530db1b00>

In [21]:
res

<tf.Tensor: shape=(1, 125, 999), dtype=float32, numpy=
array([[[9.9994636e-01, 3.1321491e-08, 2.5471282e-08, ...,
         6.3138295e-08, 7.0284877e-08, 8.5961194e-09],
        [9.9997056e-01, 2.9704390e-08, 1.1387958e-08, ...,
         5.5829531e-08, 1.4294294e-07, 1.7697323e-08],
        [9.9997270e-01, 2.6858727e-08, 1.2815090e-08, ...,
         5.8366968e-08, 1.6226754e-07, 1.6651661e-08],
        ...,
        [9.9999344e-01, 3.9918651e-09, 2.3550154e-08, ...,
         1.2114496e-07, 5.8528911e-09, 1.3914647e-07],
        [9.9998605e-01, 8.0678220e-09, 6.5247665e-09, ...,
         3.5095120e-08, 2.3782537e-08, 1.3496430e-08],
        [9.9998164e-01, 1.0532155e-08, 9.0227656e-09, ...,
         3.8783000e-08, 3.3846618e-08, 1.5370622e-08]]], dtype=float32)>

In [5]:
decoder

In [6]:
# construct a shape[1,136,999] tensor where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 0
the_tensor = []
for i in range(136):
  if i == value:
    the_tensor.append(list(np.ones(999)))
  else:
    the_tensor.append(list(np.zeros(999)))

the_tensor = np.array(the_tensor)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)


tf.Tensor(
[[[1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)


In [7]:
#res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(the_tensor.numpy())[0]))

# Test that the 999 does refer to time, and thus halving it will shorten the output
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [8]:
res1, res2 = tf.split(res, [500,499],2)
print(res1.shape)
#print(decoder(torch.Tensor(res1.numpy())[0]))
'''---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-617167afc803> in <module>()
      1 res1, res2 = tf.split(res, [500,499],2)
      2 print(res1.shape)
----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))

1 frames
/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)
     62     def process(self,
     63                 probs, wav_len, word_align):
---> 64         assert len(self.labels) == probs.shape[1]
     65         for_string = []
     66         argm = torch.argmax(probs, axis=1)

AssertionError: '''

(1, 136, 500)


'---------------------------------------------------------------------------\nAssertionError                            Traceback (most recent call last)\n<ipython-input-38-617167afc803> in <module>()\n      1 res1, res2 = tf.split(res, [500,499],2)\n      2 print(res1.shape)\n----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))\n\n1 frames\n/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)\n     62     def process(self,\n     63                 probs, wav_len, word_align):\n---> 64         assert len(self.labels) == probs.shape[1]\n     65         for_string = []\n     66         argm = torch.argmax(probs, axis=1)\n\nAssertionError: '

In [9]:
# construct a shape[1,136,999] tensore where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 11
the_tensor = []
for i in range(999):
  if i == value:
    temp = list(np.ones(136))
    temp[10] = 0
    the_tensor.append(temp)
  else:
    temp = list(np.zeros(136))
    the_tensor.append(temp)
    if i == 12:
      temp[10] = 1


the_tensor = np.array(the_tensor)
the_tensor = np.swapaxes(the_tensor, 0, 1)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)

print(decoder(torch.Tensor(the_tensor.numpy())[0]))

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)
entoen


In [10]:
#@title Install dependencies

!pip install -q omegaconf
!pip install -q torchaudio
!pip install -q soundfile
!pip install -q pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from utils import (init_jit_model, 
                   split_into_batches,
                   read_audio,
                   read_batch,
                   prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


     |████████████████████████████████| 776.8MB 21kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
/content/silero-models


In [19]:
sample_rate = 16000

clear_output()
audio = record_audio(10)
wavfile.write('/content/recorded.wav', sample_rate, (32767*audio).numpy().astype(np.int16))


#button = widgets.Button(description="Record Speech")
#button.on_click(_record_audio)
#display(button)

display(Audio(audio, rate=sample_rate, autoplay=True))
wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [12]:
len(decoder.labels)

999

In [13]:
decoder.blank_idx

0

In [47]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'______________you  would   not   _believe_   your  _eyes_____________________with   ten_ millillion  _fire__f__es______________________________________________'

In [31]:
result = np.argmax(res_array, axis=2)

#result = f(result)

In [32]:
f(3)

'in'

'______________you  would   not   _believe_   your  _eyes_____________________with   ten_ millillion  _fire__f__es______________________________________________'

In [38]:
result[0].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 37,
 998,
 998,
 184,
 998,
 998,
 998,
 90,
 998,
 998,
 998,
 0,
 933,
 0,
 998,
 998,
 998,
 215,
 998,
 998,
 0,
 681,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 63,
 998,
 998,
 998,
 429,
 0,
 998,
 982,
 104,
 104,
 39,
 998,
 998,
 0,
 984,
 325,
 0,
 0,
 984,
 0,
 0,
 21,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [68]:
#Testing singing
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'_________________________what   not_   _believe_  or_    _eyes______  if  can_  millillion  _f__ire_  _fli_es______  my    up   the all_  as i   fell as__ly_____________________________________becauseuse  they_  foundll  they   open      _air_____   and leave    your drorops  every____where    you   make_  me   will   but_   i  would  just   stand_  and________ __star___________________________i_   like_    make_  my_____self_______________________________________________________________________________________________________ it   are_  to say   that i   rathertherther________ _________ _i________ly___________ _becauseuse  _every_______   is   never_______  to  _see____________________________________ ___  i  get   a  thousand___  _hour___ves______ from   ten_ _thousand__  _light___ning_  _lo__ves__   is  they   tried  to teach_  me_   out__    d______________________________________a ho__ str_ong  ababove   i________'

In [51]:
decoder.blank_idx

0

In [ ]:
np.argsort(res_array, axis=2)[0][100]
#result = np.argmax(res_array, axis=2)
#result[0][100]

In [86]:



#Testing singing with empty removed
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

original = result[0].copy()
for i,val in enumerate(original):
  #print(val)
  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


"spspspspspspiispiiiiiiiii   ii  what   not     believeave  or      eyeseyes'eds    if  can   millillion   f ireire   ffliesesededss    my    up   the allall  as i   fell as  ly avevedveve    iiiiii22iiiiiiii222ii2222   becauseuse  they   foundll  they   open       airirirs     and leave    your drorops  everyevery   where    you   makeak  me   will   but    i  would  just   stand   and'nsss      starirss   sss iiiiiiiiiii22    i    like     make   mymymymy selfselfll   i inginginging i2222222222222    it   and   it a    i2222222ss  iiiii2sss2222    ssolowwve  ohohoh2 ohohoh22222i2222222   it   are   to say   that i   rathertherthersssss     why ingig      i m  less   lyeeingitiiii'2   becauseuse   everyevery  thth     is   nevernever  atactt   to   seeinginginginginging22222222222amanssssiiiiiiiiiii2    but   i  get   a  thousandthousands    hourouvesvesvesvessss   from   ten   thousandthousand    lightininenning    lovesvesves     is  they   tried  to teach   me    outout     d nsss 

In [89]:



#Testing singing with empty removed
speech_file = '/content/jepsen_voc.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

#original = result[0].copy()
#for i,val in enumerate(original):
  #print(val)
#  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
#    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


"____maybebe  's  right____________  maybebe there_  is  something  the  mat_ter with me_________________  i  _just   i don 't see how   world  that make   such  _wonderderful  things_____  could  be bad______  look  at  this  stuff_______  isn t  any  me_________  would     _think_    my_  _col_lect__ions_   comple__te_______  wouldn 't   _think_        girl________    _good_  who  _have____________   _every___thing________________  look_  at this_  _twelve____  tre_asas_ures_  and   all_______  _how   many  _won_dders_   can   want_  ca_verver__  who_____   look_      around_  you   you  think_________ _sure_________   she 's got_    _every___thingthing____________      get  getad  to  think  is    _   of plent_y_________ i 've got   who      from   what's it'  go _______________  you  want thing  over  us__________  i   got  _twent_y____________ _but_  who  _care__s____________  no_  big_  do_al_______________  i    want____  _________________________________________________________